In [12]:
import pandas as pd
import numpy as np

import json
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMRegressor

In [2]:
train_data = pd.read_csv('train_modified.csv') # Important!!
print(train_data.shape)
train_data.head()

/home/tintash/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(903653, 56)


,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [3]:
#Print all the column names
print(train_data.columns.values)

['Unnamed: 0' 'channelGrouping' 'date' 'fullVisitorId' 'sessionId'
 'socialEngagementType' 'visitId' 'visitNumber' 'visitStartTime'
 'device.browser' 'device.browserSize' 'device.browserVersion'
 'device.deviceCategory' 'device.flashVersion' 'device.isMobile'
 'device.language' 'device.mobileDeviceBranding' 'device.mobileDeviceInfo'
 'device.mobileDeviceMarketingName' 'device.mobileDeviceModel'
 'device.mobileInputSelector' 'device.operatingSystem'
 'device.operatingSystemVersion' 'device.screenColors'
 'device.screenResolution' 'geoNetwork.city' 'geoNetwork.cityId'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.latitude'
 'geoNetwork.longitude' 'geoNetwork.metro' 'geoNetwork.networkDomain'
 'geoNetwork.networkLocation' 'geoNetwork.region'
 'geoNetwork.subContinent' 'totals.bounces' 'totals.hits'
 'totals.newVisits' 'totals.pageviews' 'totals.transactionRevenue'
 'totals.visits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsC

In [4]:
relevantAttributes = ['totals.hits', 'visitNumber', 'totals.pageviews', 'geoNetwork.country', 'device.operatingSystem',
                      'channelGrouping',
                      'totals.transactionRevenue']

train_data['totals.transactionRevenue'] = train_data['totals.transactionRevenue'].fillna(0)
train_data['totals.pageviews'] = train_data['totals.pageviews'].fillna(1)

trainData = train_data.sample(frac=0.8,random_state=200)
testData = train_data.drop(trainData.index)

trainData_LGB = train_data[relevantAttributes]
testData_LGB = testData[relevantAttributes]

trainData_LGB = pd.DataFrame(trainData_LGB)
testData_LGB = pd.DataFrame(testData_LGB)

In [5]:
#Figure out labeling for country param
sortedCountries = train_data.groupby('geoNetwork.country')['totals.transactionRevenue'].mean()
sortedCountries = sortedCountries.sort_values(ascending=False)
countryCodeDict = {}
i = 0
for country in sortedCountries.index:
    countryCodeDict[country] = i
    i += 1
    
sortedDevices = train_data.groupby('device.operatingSystem')['totals.transactionRevenue'].mean()
sortedDevices = sortedDevices.sort_values(ascending=False)
deviceCodeDict = {}
i = 0
for device in sortedDevices.index:
    deviceCodeDict[device] = i
    i += 1

sortedChannelGrouping = train_data.groupby('channelGrouping')['totals.transactionRevenue'].mean()
sortedChannelGrouping = sortedChannelGrouping.sort_values(ascending=False)
channelGroupingCodeDict = {}
i = 0
for device in sortedChannelGrouping.index:
    channelGroupingCodeDict[device] = i
    i += 1

print(channelGroupingCodeDict)
print(deviceCodeDict)

{'Display': 0, 'Referral': 1, 'Direct': 2, 'Paid Search': 3, 'Organic Search': 4, '(Other)': 5, 'Affiliates': 6, 'Social': 7}
{'Chrome OS': 0, 'Macintosh': 1, 'Linux': 2, 'Windows': 3, 'Android': 4, 'iOS': 5, 'Windows Phone': 6, 'NTT DoCoMo': 7, 'BlackBerry': 8, 'Firefox OS': 9, 'FreeBSD': 10, 'Nintendo 3DS': 11, 'Xbox': 12, 'Nintendo Wii': 13, 'Nintendo WiiU': 14, 'Nokia': 15, 'OpenBSD': 16, 'Samsung': 17, 'SunOS': 18, '(not set)': 19}


In [6]:
trainY = trainData_LGB["totals.transactionRevenue"]
trainX = trainData_LGB[['totals.hits', 'visitNumber', 'totals.pageviews', 'geoNetwork.country'
                            , 'device.operatingSystem', 'channelGrouping']]

trainX['geoNetwork.country'] = trainX['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
trainX['device.operatingSystem'] = trainX['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
trainX['channelGrouping'] = trainX['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])

testY = testData_LGB["totals.transactionRevenue"]
testX = testData_LGB[['totals.hits', 'visitNumber', 'totals.pageviews', 'geoNetwork.country'
                            , 'device.operatingSystem', 'channelGrouping']]

testX['geoNetwork.country'] = testX['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
testX['device.operatingSystem'] = testX['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
testX['channelGrouping'] = testX['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])


/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [7]:
# trainY = np.log1p(trainY.astype(float))
train_x, val_x, train_y, val_y = train_test_split(trainX, trainY, test_size=0.25, random_state=20)

# Feature Scaling
sc = StandardScaler()
train_x = sc.fit_transform(train_x)
testX = sc.transform(testX)

In [8]:

# X = X[['totals.hits', 'visitNumber', 'totals.pageviews', 'geoNetwork.country',
#                            'device.operatingSystem']]

# params = {}
# params['learning_rate'] = 0.03
# params['boosting_type'] = 'gbdt'
# params['objective'] = 'regression'
# params['metric'] = 'rmse'
# params['categorical_feature'] = 3,4,5
# params['num_leaves'] = 36
# params['min_data'] = 50
# params['bagging_fraction'] = 0.75
# params['feature_fraction'] = 0.6
# params['bagging_frequency'] = 7

# lgb_params = {"objective" : "regression", "metric" : "rmse",
#               "num_leaves" : 36, "learning_rate" : 0.05, "bagging_fraction" : 0.75, "feature_fraction" : 0.6, "bagging_frequency" : 7}

# # d_train = lgb.Dataset(trainX, label=trainY)
# lgb_train = lgb.Dataset(train_x, label=train_y)
# lgb_val = lgb.Dataset(val_x, label=val_y)
# model = lgb.train(lgb_params, lgb_train, 300, valid_sets=[lgb_val], early_stopping_rounds=50, verbose_eval=100)
# # clf = lgb.train(lgb_params, d_train, 300)



#Initialize LGBM
reg = LGBMRegressor(objective = 'regression', 
                     boosting_type = 'gbdt', 
                     metric = 'rmse',
                     n_estimators = 10000, #10000
                     num_leaves = 36,
                     learning_rate = 0.01, #0.01
                     bagging_fraction = 0.9,#0.8
                     feature_fraction = 0.3,#.3
                     #bagging_seed = 42,
                     #max_depth = 12, #-1 
                     #categorical_feature = [train_df[cat_cols].columns.get_loc(c) for c in cat_cols ] ,
                     #cat_smooth = 20
                    )

reg.fit(
    train_x, np.log1p(train_y),
    eval_set=[(val_x, np.log1p(val_y))],
    early_stopping_rounds=50,
    verbose=100,
    eval_metric='rmse'
)

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[28]	valid_0's rmse: 1.98702


LGBMRegressor(bagging_fraction=0.9, boosting_type='gbdt', class_weight=None,
       colsample_bytree=1.0, feature_fraction=0.3, importance_type='split',
       learning_rate=0.01, max_depth=-1, metric='rmse',
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=10000, n_jobs=-1, num_leaves=30,
       objective='regression', random_state=None, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [9]:
#Prediction
y_pred=reg.predict(testX , num_iteration=reg.best_iteration_)

#convert into binary values

# for i in range(0,99):
#     if y_pred[i]>=.5:       # setting threshold to .5
#         y_pred[i]=1
#     else:
#         y_pred[i]=0
        

In [10]:
y = testY

y_pred = y_pred + 1
y = y + 1

y_pred_ln = np.log(y_pred)
y_true_ln = np.log(y)

difference = np.subtract(y_pred_ln, y_true_ln)

sqr = np.square(difference)

sumSqr = sqr.sum()

sumSqrMean = sumSqr/len(y)

sumSqrRootMean = np.sqrt(sumSqrMean)
sumSqrRootMean

1.9727944522361966

In [ ]:
# params = {}
# params['learning_rate'] = 0.003
# params['boosting_type'] = 'gbdt'
# params['objective'] = 'binary'
# params['metric'] = 'binary_logloss'
# params['sub_feature'] = 0.5
# params['num_leaves'] = 10
# params['min_data'] = 50
# params['max_depth'] = 10